# Finding open security groups with the AWS API

AWS API Documentation: https://boto3.amazonaws.com/v1/documentation/api/latest/index.html

In this exercise we'll start digging into the AWS API and learn how to find security groups that are open to the world and the EC2 instances that use them.

Steps:
1) Find all open security groups and output them in an easy-to-use format
2) Find of the security groups used by running instances
3) Check a whitelist to see whether each instance should be allowed to use an open security group
4) Disable non-whitelisted security groups 
5) Send a notification to be logged in Splunk

In [1]:
# First we'll import boto3 to be used for everything else
import boto3

## describe_security_groups
First we'll take a look at what visibility AWS offers us into security group configurations. The `describe_security_group` endpoint can return information for all security groups, giving us a good starting point.

Documentation: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ec2.html#EC2.Client.describe_security_groups

In [2]:
def open_security_groups():
    """Return all security groups that are allow inbound connections from the """
    ec2_client = boto3.client("ec2")
    security_groups = ec2_client.describe_security_groups(
        Filters=[
            {"Name": "ip-permission.cidr", "Values": ["0.0.0.0/0"]},
        ]
    )
    # FIXME: Apply whitelist here to allow port 443
    return {sg["GroupId"] : sg for sg in security_groups["SecurityGroups"]}

open_security_groups()

{'sg-05777ecea90c47aae': {'Description': 'launch-wizard-2 created 2020-01-26T15:09:59.654-08:00',
  'GroupName': 'launch-wizard-2',
  'IpPermissions': [{'FromPort': 8080,
    'IpProtocol': 'tcp',
    'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
    'Ipv6Ranges': [{'CidrIpv6': '::/0'}],
    'PrefixListIds': [],
    'ToPort': 8080,
    'UserIdGroupPairs': []},
   {'FromPort': 22,
    'IpProtocol': 'tcp',
    'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
    'Ipv6Ranges': [],
    'PrefixListIds': [],
    'ToPort': 22,
    'UserIdGroupPairs': []}],
  'OwnerId': '583449068983',
  'GroupId': 'sg-05777ecea90c47aae',
  'IpPermissionsEgress': [{'IpProtocol': '-1',
    'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
    'Ipv6Ranges': [],
    'PrefixListIds': [],
    'UserIdGroupPairs': []}],
  'VpcId': 'vpc-6abc2e12'},
 'sg-078a6a1cfe4d82489': {'Description': 'Managed by Terraform',
  'GroupName': 'allow_automation_ports_ingress',
  'IpPermissions': [{'FromPort': 80,
    'IpProtocol': 'tcp',
    'IpRanges': [{'CidrIp

In [6]:
def instance_security_groups():
    """Return a summary of all the security group IDs assocated with running instances.
    
    This format will look like: 
        {'i-096e3b9655241f365': ['sg-05777ecea90c47aae'], ...}
    """
    ec2_client = boto3.client("ec2")
    running_instances = ec2_client.describe_instances(
        Filters=[
            {"Name": "instance-state-name", "Values": ["running"]},
        ]
    )
    instances = {}
    for reservation in running_instances["Reservations"]:
        for instance in reservation["Instances"]:
            instance_id = instance["InstanceId"]
            for iface in instance["NetworkInterfaces"]:
                instances[instance_id] = []
                for group in iface["Groups"]:
                    instances[instance_id].append(group["GroupId"])
    return instances
    
instance_security_groups()

{'i-096e3b9655241f365': ['sg-05777ecea90c47aae']}

Now that we have easier-to-parse lists of all security groups and the security groups our instances use, we can go through them and easily find which instances are using open security groups.

In [7]:
open_groups = open_security_groups()
instance_groups = instance_security_groups()
for instance, groups in instance_groups.items():
    intersection = set(groups).intersection(open_groups)
    print(instance, intersection)

i-096e3b9655241f365 {'sg-05777ecea90c47aae'}


Now that we know we can identify instances with open groups, let's write the code to remove that open group

In [12]:
def remove_security_group(instance_id, sg_id):
    ec2_client = boto3.resource("ec2")
    instance = ec2_client.Instance(instance_id)
    instance.modify_attribute(Groups=[g for g in instance.security_groups if g['GroupId'] not sg_id]) 

remove_security_group('i-096e3b9655241f365', 'sg-05777ecea90c47aae ')

[{'GroupName': 'launch-wizard-2', 'GroupId': 'sg-05777ecea90c47aae'}]

Now we can put it all together:

In [ ]:
def remediate_security_groups():
    open_groups = open_security_groups()
    instance_groups = instance_security_groups()
    for instance, groups in instance_groups.items():
        intersection = set(groups).intersection(open_groups)
    remove_security_group(instance, intersection)